In [ ]:
# ==============================================================================
# CELL 1: SIMPLE DATA PREPARATION
# ==============================================================================
import os
import shutil
import torch
import numpy as np
from PIL import Image
from tqdm import tqdm
from scipy.ndimage import label as cc_label
from google.colab import drive

# 1. MOUNT
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive', force_remount=True)

# 2. PATHS
DRIVE_DATA = "/content/drive/MyDrive/Latitude_AI_Team/data"
LOCAL_DATA = "/content/data"

# Clear local disk
if os.path.exists(LOCAL_DATA): shutil.rmtree(LOCAL_DATA)

# Create directories
PT_DIR = os.path.join(LOCAL_DATA, "precomputed_masks")
IMG_DIR = os.path.join(LOCAL_DATA, "images")
os.makedirs(PT_DIR, exist_ok=True)
os.makedirs(os.path.join(IMG_DIR, "train"), exist_ok=True)
os.makedirs(os.path.join(IMG_DIR, "val"), exist_ok=True)

# Config
BDD_IDS = [6, 7, 11, 13, 12, 14, 15, 17, 18]
ID_TO_LABEL = {bdd_id: i+1 for i, bdd_id in enumerate(BDD_IDS)}

# 3. PROCESSING FUNCTION
def process_split(mask_folder, img_folder, split_name):
    print(f"\n Processing {split_name}...")
    mask_source = os.path.join(DRIVE_DATA, mask_folder)
    img_source = os.path.join(DRIVE_DATA, img_folder)

    files = [f for f in os.listdir(mask_source) if f.endswith('.png')]

    for f in tqdm(files):
        try:
            basename = f.replace("_train_id.png", "")

            # Load Mask
            mask = np.array(Image.open(os.path.join(mask_source, f)).convert("L"), dtype=np.int32)

            boxes, labels, masks = [], [], []

            # Extract Instances
            for bdd_id in BDD_IDS:
                class_mask = (mask == bdd_id).astype(np.uint8)
                if class_mask.sum() < 10: continue

                inst_map, ninst = cc_label(class_mask)
                for i in range(1, ninst+1):
                    pos = np.where(inst_map == i)
                    ymin, xmin, ymax, xmax = np.min(pos[0]), np.min(pos[1]), np.max(pos[0]), np.max(pos[1])
                    if (ymax-ymin) < 5 or (xmax-xmin) < 5: continue

                    boxes.append([xmin, ymin, xmax, ymax])
                    labels.append(ID_TO_LABEL[bdd_id])
                    masks.append((inst_map == i).astype(np.uint8))

            if not boxes: continue

            # Save PT
            torch.save({
                "boxes": torch.tensor(boxes, dtype=torch.float32),
                "labels": torch.tensor(labels, dtype=torch.int64),
                "masks": torch.tensor(np.array(masks), dtype=torch.uint8)
            }, os.path.join(PT_DIR, basename + ".pt"))

            # Copy Image
            src_img = os.path.join(img_source, basename + ".jpg")
            dst_img = os.path.join(IMG_DIR, split_name, basename + ".jpg")
            if os.path.exists(src_img): shutil.copy(src_img, dst_img)

        except: pass

# 4. EXECUTE
# Train Set
process_split(
    "bdd100k_segmentations/bdd100k_seg_maps_2/labels/train",
    "bdd100k_images_10k/10k/train",
    "train"
)

# Validation Set
process_split(
    "bdd100k_segmentations/bdd100k_seg_maps_2/labels/val",
    "bdd100k_images_10k/10k/val",
    "val"
)

# 5. BACKUP
print("\n📦 Zipping...")
shutil.make_archive("/content/bdd_cache", 'zip', LOCAL_DATA)
shutil.move("/content/bdd_cache.zip", os.path.join(DRIVE_DATA, "bdd100k_fast_cache.zip"))
print("✅ DONE.")

Mounted at /content/drive

🚀 Processing train...


100%|██████████| 7000/7000 [39:15<00:00,  2.97it/s]



🚀 Processing val...


100%|██████████| 1000/1000 [06:03<00:00,  2.75it/s]



📦 Zipping...
✅ DONE.


In [4]:
# ==============================================================================
# CELL 2: FULL PRODUCTION TRAINING RUN (25 EPOCHS)

!pip install torchmetrics wandb -q

# IMPORTS
import os
import shutil
import time
import random
import torch
import wandb
import torchvision
import torchvision.transforms as T
import torchvision.transforms.functional as F
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchmetrics.detection import MeanAveragePrecision
from torch.cuda import empty_cache

wandb.login()

# CONFIGURATION

DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"🚀 Training on: {DEVICE}")

NUM_WORKERS = 2         # Fast I/O for local data
PIN_MEMORY = True

# Training Settings
BATCH_SIZE = 2
EPOCHS = 25
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 0.0005

# Local Data Paths
DATA_ROOT = "/content/data"
PT_DIR = os.path.join(DATA_ROOT, "precomputed_masks")
TRAIN_IMG_DIR = os.path.join(DATA_ROOT, "images/train")
VAL_IMG_DIR = os.path.join(DATA_ROOT, "images/val")

# Persistence Paths (Google Drive)
DRIVE_MODELS_DIR = "/content/drive/MyDrive/Latitude_AI_Team/models"
os.makedirs(DRIVE_MODELS_DIR, exist_ok=True)

CHECKPOINT_FILENAME = "best_model_checkpoint_full_run.pth"
LATEST_FILENAME = "latest_model_checkpoint.pth"

# Full Paths
DRIVE_CHECKPOINT_PATH = os.path.join(DRIVE_MODELS_DIR, CHECKPOINT_FILENAME)
DRIVE_LATEST_PATH = os.path.join(DRIVE_MODELS_DIR, LATEST_FILENAME)
LOCAL_CHECKPOINT_PATH = CHECKPOINT_FILENAME
LOCAL_LATEST_PATH = LATEST_FILENAME

# DATA INTEGRITY CHECK
if not os.path.exists(PT_DIR) or len(os.listdir(PT_DIR)) == 0:
    raise FileNotFoundError(f"CRITICAL: Local data missing at {PT_DIR}")

# 4. DATASET CLASS
class BDDPrecomputedDataset(Dataset):
    def __init__(self, img_dir, pt_dir, basenames, transforms=None, train_mode=False):
        self.img_dir = img_dir
        self.pt_dir = pt_dir
        self.basenames = basenames
        self.transforms = transforms
        self.train_mode = train_mode
        self.color_jitter = T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05)

    def __len__(self): return len(self.basenames)

    def __getitem__(self, idx):
        basename = self.basenames[idx]

        # Load Image (Fast from local SSD)
        img_path = os.path.join(self.img_dir, basename + ".jpg")
        img = Image.open(img_path).convert("RGB")

        # Load Tensors
        data = torch.load(os.path.join(self.pt_dir, basename + ".pt"))

        boxes = data["boxes"]
        labels = data["labels"]
        masks = data["masks"]

        # Augmentation
        if self.train_mode:
            if random.random() < 0.5: img = self.color_jitter(img)
            if random.random() < 0.5:
                img = F.hflip(img)
                masks = F.hflip(masks)
                W_img, _ = img.size
                boxes[:, [0, 2]] = W_img - boxes[:, [2, 0]]

        target = {"boxes": boxes, "labels": labels, "masks": masks, "image_id": torch.tensor([idx], dtype=torch.int64)}
        img_tensor = T.ToTensor()(img)
        return img_tensor, target

def collate_fn(batch): return tuple(zip(*batch))


# Scan local folder to define splits based on actual files present
all_pt = set(f.replace('.pt', '') for f in os.listdir(PT_DIR) if f.endswith('.pt'))
train_imgs = set(f.replace('.jpg', '') for f in os.listdir(TRAIN_IMG_DIR))
val_imgs = set(f.replace('.jpg', '') for f in os.listdir(VAL_IMG_DIR))

# Intersect to ensure we only load valid pairs
train_basenames = sorted(list(all_pt.intersection(train_imgs)))
val_basenames = sorted(list(all_pt.intersection(val_imgs)))

TRAIN_SIZE = len(train_basenames)
VAL_SIZE = len(val_basenames)

print(f" Dataset: {TRAIN_SIZE} Training | {VAL_SIZE} Validation")

train_loader = DataLoader(
    BDDPrecomputedDataset(TRAIN_IMG_DIR, PT_DIR, train_basenames, train_mode=True),
    batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, collate_fn=collate_fn, pin_memory=PIN_MEMORY
)
val_loader = DataLoader(
    BDDPrecomputedDataset(VAL_IMG_DIR, PT_DIR, val_basenames, train_mode=False),
    batch_size=1, shuffle=False, num_workers=NUM_WORKERS, collate_fn=collate_fn, pin_memory=PIN_MEMORY
)

# METRICS
def calculate_ap_metrics(model, val_loader, device, epoch):
    model.eval()
    # Use 'segm' for the full quality check on masks
    metric = MeanAveragePrecision(box_format="xyxy", iou_type="segm").to(device)

    with torch.no_grad():
        for i, (images, targets) in enumerate(val_loader):
            images = [img.to(device) for img in images]
            outputs = model(images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            processed = []
            for pred in outputs:
                if 'masks' in pred:
                    # Convert float masks to uint8
                    pred['masks'] = (pred['masks'] > 0.5).squeeze(1).to(torch.uint8)
                processed.append(pred)
            metric.update(processed, targets)

    res = metric.compute()
    mAP_all = res['map'].item(); mAP_50 = res['map_50'].item(); mAP_75 = res['map_75'].item()

    wandb.log({
        "val/mAP_iou_all": mAP_all,
        "val/mAP_iou_50": mAP_50,
        "val/mAP_iou_75": mAP_75,
        "val/mAP_small": res['map_small'].item(),
        "epoch": epoch
    })
    model.train(); empty_cache()
    return mAP_all

# MODEL & OPTIMIZER
BDD_IDS = [6, 7, 11, 13, 12, 14, 15, 17, 18]; NUM_CLASSES = 1 + len(BDD_IDS)

def get_model(num_classes):
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, 256, num_classes)
    return model

model = get_model(NUM_CLASSES).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3)

# RESUMPTION (From Drive)
start_epoch = 1; best_avg_ap = 0.0

# Check Drive for checkpoints (since local is fresh)
if os.path.exists(DRIVE_LATEST_PATH):
    print(f" Found LATEST checkpoint on Drive. Copying...")
    shutil.copy(DRIVE_LATEST_PATH, LOCAL_LATEST_PATH)
    ckpt = torch.load(LOCAL_LATEST_PATH)
    model.load_state_dict(ckpt['model_state_dict'])
    optimizer.load_state_dict(ckpt['optimizer_state_dict'])
    start_epoch = ckpt['epoch'] + 1
    best_avg_ap = ckpt.get('mAP', 0.0)
    print(f" Resuming from Epoch {start_epoch}. Best AP: {best_avg_ap:.4f}")
elif os.path.exists(DRIVE_CHECKPOINT_PATH):
    print(f" Found BEST checkpoint on Drive. Copying...")
    shutil.copy(DRIVE_CHECKPOINT_PATH, LOCAL_CHECKPOINT_PATH)
    ckpt = torch.load(LOCAL_CHECKPOINT_PATH)
    model.load_state_dict(ckpt['model_state_dict'])
    optimizer.load_state_dict(ckpt['optimizer_state_dict'])
    start_epoch = ckpt['epoch'] + 1
    best_avg_ap = ckpt.get('mAP', 0.0)
    print(f"Resuming from Epoch {start_epoch} (Best Model).")

wandb.init(project="latitude-ai-maskrcnn", name="FULL-PRODUCTION-RUN", config={
    "epochs": EPOCHS, "train_size": TRAIN_SIZE, "val_size": VAL_SIZE, "lr": LEARNING_RATE
})

# TRAINING LOOP
print(f" Starting Full Training Loop ({EPOCHS} Epochs)...")
global_step = 0
model.train()

for epoch in range(start_epoch, EPOCHS + 1):
    epoch_loss = 0.0
    start = time.time()

    # Track detailed loss components
    running_losses = {"mask": 0, "box": 0, "class": 0, "obj": 0, "rpn_box": 0}

    for i, (images, targets) in enumerate(train_loader):
        images = [img.to(DEVICE) for img in images]
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        total_loss = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad(); total_loss.backward(); optimizer.step()
        epoch_loss += total_loss.item(); global_step += 1

        # Accumulate components
        running_losses["mask"] += loss_dict['loss_mask'].item()
        running_losses["box"] += loss_dict['loss_box_reg'].item()
        running_losses["class"] += loss_dict['loss_classifier'].item()
        running_losses["obj"] += loss_dict['loss_objectness'].item()
        running_losses["rpn_box"] += loss_dict['loss_rpn_box_reg'].item()

        # Log all 5 Losses
        wandb.log({
            "train/total_loss": total_loss.item(),
            "train/loss_mask": loss_dict['loss_mask'].item(),
            "train/loss_box_reg": loss_dict['loss_box_reg'].item(),
            "train/loss_classifier": loss_dict['loss_classifier'].item(),
            "train/loss_objectness": loss_dict['loss_objectness'].item(),
            "train/loss_rpn_box_reg": loss_dict['loss_rpn_box_reg'].item(),
            "epoch": epoch, "step": global_step, "lr": optimizer.param_groups[0]['lr']
        })

    avg_loss = epoch_loss / len(train_loader)
    num_batches = len(train_loader)

    # Print Average Components
    print(f"\n{'='*60}")
    print(f"EPOCH {epoch}/{EPOCHS} COMPLETE | Time: {(time.time()-start)/60:.1f} min")
    print(f"{'='*60}")
    print(f"📉 Total Loss: {avg_loss:.4f}")
    print(f"   • Mask Loss:       {running_losses['mask']/num_batches:.4f}")
    print(f"   • Box Reg Loss:    {running_losses['box']/num_batches:.4f}")
    print(f"   • Classifier Loss: {running_losses['class']/num_batches:.4f}")
    print(f"   • Objectness Loss: {running_losses['obj']/num_batches:.4f}")
    print(f"   • RPN Box Loss:    {running_losses['rpn_box']/num_batches:.4f}")

    print(f" Validating Epoch {epoch}...")
    ap = calculate_ap_metrics(model, val_loader, DEVICE, epoch)
    lr_scheduler.step(ap)

    print(f" Validation AP: {ap:.4f} (Best: {best_avg_ap:.4f})")
    print(f"{'='*60}\n")

    if ap > best_avg_ap:
        best_avg_ap = ap
        torch.save({
            'epoch': epoch, 'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(), 'mAP': best_avg_ap
        }, LOCAL_CHECKPOINT_PATH)
        shutil.copy(LOCAL_CHECKPOINT_PATH, DRIVE_CHECKPOINT_PATH) # Sync to Drive
        print(f"   ->  Saved NEW BEST Checkpoint (AP {ap:.4f}) to Drive")

        artifact = wandb.Artifact(f'model-epoch-{epoch}', type='model')
        artifact.add_file(LOCAL_CHECKPOINT_PATH)
        wandb.log_artifact(artifact)

    # Save Latest (Safety)
    torch.save({
        'epoch': epoch, 'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(), 'mAP': ap
    }, LOCAL_LATEST_PATH)
    shutil.copy(LOCAL_LATEST_PATH, DRIVE_LATEST_PATH) # Sync to Drive
    print("   ->  Saved Safety Backup to Drive")

wandb.finish()
print(" Full Training Pipeline Completed Successfully.")

🚀 Training on: cuda
📊 Dataset: 6945 Training | 988 Validation


🚀 Starting Full Training Loop (25 Epochs)...

✅ EPOCH 1/25 COMPLETE | Time: 39.7 min
📉 Total Loss: 1.1373
   • Mask Loss:       0.2561
   • Box Reg Loss:    0.3114
   • Classifier Loss: 0.2665
   • Objectness Loss: 0.1034
   • RPN Box Loss:    0.2000
⏳ Validating Epoch 1...
📈 Validation AP: 0.1814 (Best: 0.0000)

   -> 💾 Saved NEW BEST Checkpoint (AP 0.1814) to Drive
   -> 🛡️ Saved Safety Backup to Drive

✅ EPOCH 2/25 COMPLETE | Time: 39.8 min
📉 Total Loss: 1.0376
   • Mask Loss:       0.2398
   • Box Reg Loss:    0.2909
   • Classifier Loss: 0.2378
   • Objectness Loss: 0.0840
   • RPN Box Loss:    0.1851
⏳ Validating Epoch 2...
📈 Validation AP: 0.2012 (Best: 0.1814)

   -> 💾 Saved NEW BEST Checkpoint (AP 0.2012) to Drive
   -> 🛡️ Saved Safety Backup to Drive

✅ EPOCH 3/25 COMPLETE | Time: 39.8 min
📉 Total Loss: 0.9912
   • Mask Loss:       0.2329
   • Box Reg Loss:    0.2818
   • Classifier Loss: 0.2260
   • Objectness Loss: 0.0737
   • RPN Box Loss:    0.1768
⏳ Validating Epoch 3...

epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇█
lr,███████████████████▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇███
train/loss_box_reg,█▄▅▇▄▄▇▆▄▆▆▅▇▃▆▆▄▄▃▅▃▅█▃▆▂▄▃▅▃▃▅▃▂▄▄▁▅▄▂
train/loss_classifier,▇▂▄▇▅▃█▆▅▇▅▆▄▄▄▂▄▆▃▃▇▅▃▂▃▂▃▂▄▂▂▂▅▃▂▂▁▂▂▃
train/loss_mask,▄▄▃▅▅▃▂▄▄▃▂▃▄▅▅▄▁▄█▆▅▂▄▄▃▃▁▃▃▃▃▄▃▂▄▄▃▁▃▄
train/loss_objectness,█▃▆▂▂▃▂▂▃▂▁▂▂▂▂▂▁▁▃▂▁▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_rpn_box_reg,▂█▅▄▅▃▃▃▂▃▃▆▂▄▃▁▄▄▄▂▃▃▄▂▂▂▂▄▁▅▃▂▁▁▂▄▁▂▁▂
train/total_loss,▇▇▃█▄▅▃▄▅▄▃▅▄▄▅▅▄▇▃▂▆▁▄▂▂▂▃▄▂▄▁▃▅▁▂▃▃▃▂▁
val/mAP_iou_50,▁▃▄▅▅▅▇█▆▇▇▇██▇▇▇▇▇▇▇▇▇▇▇
+3,...


🎉 Full Training Pipeline Completed Successfully.
